In [ ]:
import os
import io
import contextlib
from dotenv import load_dotenv

load_dotenv()

In [ ]:
GOOGLE_GEMINI_API_KEY = os.environ.get("GOOGLE_GEMINI_API_KEY")
assert GOOGLE_GEMINI_API_KEY is not None

In [ ]:
BRIGHT_DATA_SERP_API_KEY = os.environ.get("BRIGHT_DATA_SERP_API_KEY")
assert BRIGHT_DATA_SERP_API_KEY is not None

In [ ]:
from langchain_brightdata import BrightDataSERP

serp_tool = BrightDataSERP(
    bright_data_api_key=BRIGHT_DATA_SERP_API_KEY,
    search_engine="google",
    country="us",
    parse_results=True
)

In [ ]:
serp_tool.name

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.5-flash", model_provider="google_genai", api_key=GOOGLE_GEMINI_API_KEY)

In [ ]:
agent_prompt = """
You are an expert assistant that can search the internet to find the best reddit communities for any given topic.

You are also an expert at finding niche communites that discuss the same topic. 
"""

In [ ]:
from pydantic import BaseModel, Field

class RedditCommunity(BaseModel):
    name: str = Field(description="Formatted name for Reddit")
    url: str = Field(description="The complete url of the reddit community")
    subreddit_slug: str = Field(description="The slug of the subbreddit such as r/python or r/web or r/trending")
    member_count: int | None = Field(description="Current member count, if available.")

class RedditCommunites(BaseModel):
    communities: list[RedditCommunity] = Field(description="The list of reddit communites")

In [ ]:
from langgraph.prebuilt import create_react_agent

reddit_agent = create_react_agent(
    model=model,  
    tools=[serp_tool],
    prompt=agent_prompt,
    response_format=RedditCommunites
)

# Run the agent
results = reddit_agent.invoke(
    {"messages": [{"role": "user", "content": "python, javascript, django, web"}]},
    stream_mode="values"
)

In [ ]:
results['messages'][-1].content

In [ ]:
for subreddit in results["structured_response"].communities:
    print(subreddit.name, subreddit.url, subreddit.subreddit_slug, subreddit.member_count)